In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout
from keras.models import Sequential
from keras.optimizers import Adam
import tensorflow as tf

In [9]:
np.random.seed(3141593)
tf.set_random_seed(3141593)
data = pd.read_csv("../input/leaf-classification/train.csv")
data.species = data.species.astype('str').astype('category')
output_nodes = data.species.nunique()
data.id = data.id.apply(lambda x: "../input/preprocessed-images/preprocessed_images/preprocessed_images/"+str(x)+".jpg")
trainset, testset = train_test_split(data, test_size=0.1, stratify=data.species, random_state=3141593)
del data

In [5]:
onehot = LabelBinarizer()
y_train = onehot.fit_transform(trainset.species)
y_test = onehot.transform(testset.species)

In [6]:
img_size = 64
b_size = 32

train_gen = ImageDataGenerator(rescale=1/.255, width_shift_range=0.1, height_shift_range=0.1, rotation_range=0.1,
                               shear_range=0.1, zoom_range=0.1, horizontal_flip=True, vertical_flip=True)
test_gen=ImageDataGenerator(rescale=1/.255)

train_generator = train_gen.flow_from_dataframe(trainset, directory=None, x_col='id', y_col='species', 
                                                target_size=(img_size,img_size), color_mode='grayscale',
                                                class_mode='categorical', batch_size=b_size)

test_generator = test_gen.flow_from_dataframe(testset, directory=None, x_col='id', y_col='species', 
                                                target_size=(img_size,img_size), color_mode='grayscale',
                                                class_mode='categorical', batch_size=b_size)

Found 891 images belonging to 99 classes.
Found 99 images belonging to 99 classes.


In [11]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, (5,5), padding='same', strides=(1,1), activation='relu', input_shape=(64,64,1)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(32, (5,5), padding='same', strides=(1,1), activation='relu'))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
cnn_model.add(Flatten())
cnn_model.add(Dense(256, activation='relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(99, activation='softmax'))

cnn_model.compile(Adam(lr=1e-4, decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
cnn_model.fit_generator(train_generator, steps_per_epoch=10, epochs=400,
                    validation_data=test_generator, validation_steps=3)

Epoch 1/400
10/10 [==============================] - 2s 215ms/step - loss: 5.6028 - acc: 0.0187 - val_loss: 4.4845 - val_acc: 0.0149
Epoch 2/400
10/10 [==============================] - 1s 62ms/step - loss: 4.6195 - acc: 0.0312 - val_loss: 4.4328 - val_acc: 0.0625
Epoch 3/400
10/10 [==============================] - 1s 88ms/step - loss: 4.4873 - acc: 0.0313 - val_loss: 4.3363 - val_acc: 0.0746
Epoch 4/400
10/10 [==============================] - 1s 88ms/step - loss: 4.4500 - acc: 0.0281 - val_loss: 4.3658 - val_acc: 0.0299
Epoch 5/400
10/10 [==============================] - 1s 91ms/step - loss: 4.4248 - acc: 0.0469 - val_loss: 4.1766 - val_acc: 0.0746
Epoch 6/400
10/10 [==============================] - 1s 93ms/step - loss: 4.4104 - acc: 0.0443 - val_loss: 4.1554 - val_acc: 0.0729
Epoch 7/400
10/10 [==============================] - 1s 92ms/step - loss: 4.3518 - acc: 0.0750 - val_loss: 4.2911 - val_acc: 0.0597
Epoch 8/400
10/10 [==============================] - 1s 90ms/step - loss: 4

10/10 [==============================] - 1s 91ms/step - loss: 2.7848 - acc: 0.2656 - val_loss: 2.3294 - val_acc: 0.3284
Epoch 124/400
10/10 [==============================] - 1s 89ms/step - loss: 2.8158 - acc: 0.2546 - val_loss: 2.7796 - val_acc: 0.2537
Epoch 125/400
10/10 [==============================] - 1s 87ms/step - loss: 2.9637 - acc: 0.2313 - val_loss: 2.3157 - val_acc: 0.3134
Epoch 126/400
10/10 [==============================] - 1s 93ms/step - loss: 2.7688 - acc: 0.2613 - val_loss: 2.5851 - val_acc: 0.3125
Epoch 127/400
10/10 [==============================] - 1s 91ms/step - loss: 2.9072 - acc: 0.2375 - val_loss: 2.4595 - val_acc: 0.2985
Epoch 128/400
10/10 [==============================] - 1s 88ms/step - loss: 2.8255 - acc: 0.2500 - val_loss: 2.5827 - val_acc: 0.2687
Epoch 129/400
10/10 [==============================] - 1s 90ms/step - loss: 2.9063 - acc: 0.2426 - val_loss: 2.5368 - val_acc: 0.2985
Epoch 130/400
10/10 [==============================] - 1s 95ms/step - loss: 

10/10 [==============================] - 1s 93ms/step - loss: 2.0673 - acc: 0.4219 - val_loss: 1.8460 - val_acc: 0.4328
Epoch 246/400
10/10 [==============================] - 1s 95ms/step - loss: 2.2293 - acc: 0.3656 - val_loss: 1.9342 - val_acc: 0.4375
Epoch 247/400
10/10 [==============================] - 1s 85ms/step - loss: 2.1719 - acc: 0.3464 - val_loss: 1.8963 - val_acc: 0.3881
Epoch 248/400
10/10 [==============================] - 1s 93ms/step - loss: 2.0973 - acc: 0.3844 - val_loss: 1.8002 - val_acc: 0.4627
Epoch 249/400
10/10 [==============================] - 1s 92ms/step - loss: 2.0984 - acc: 0.3875 - val_loss: 1.7268 - val_acc: 0.4478
Epoch 250/400
10/10 [==============================] - 1s 89ms/step - loss: 2.2284 - acc: 0.3213 - val_loss: 1.8037 - val_acc: 0.4479
Epoch 251/400
10/10 [==============================] - 1s 89ms/step - loss: 2.0858 - acc: 0.4156 - val_loss: 1.7810 - val_acc: 0.4627
Epoch 252/400
10/10 [==============================] - 1s 90ms/step - loss: 

10/10 [==============================] - 1s 88ms/step - loss: 1.7897 - acc: 0.4283 - val_loss: 1.5582 - val_acc: 0.5672
Epoch 368/400
10/10 [==============================] - 1s 87ms/step - loss: 1.7457 - acc: 0.4562 - val_loss: 1.6413 - val_acc: 0.5075
Epoch 369/400
10/10 [==============================] - 1s 101ms/step - loss: 1.7833 - acc: 0.4750 - val_loss: 1.4086 - val_acc: 0.5821
Epoch 370/400
10/10 [==============================] - 1s 103ms/step - loss: 1.7287 - acc: 0.4627 - val_loss: 1.5537 - val_acc: 0.5104
Epoch 371/400
10/10 [==============================] - 1s 100ms/step - loss: 1.7651 - acc: 0.4250 - val_loss: 1.7062 - val_acc: 0.4776
Epoch 372/400
10/10 [==============================] - 1s 103ms/step - loss: 1.7616 - acc: 0.4594 - val_loss: 1.3606 - val_acc: 0.5373
Epoch 373/400
10/10 [==============================] - 1s 100ms/step - loss: 1.7697 - acc: 0.4497 - val_loss: 1.7052 - val_acc: 0.5522
Epoch 374/400
10/10 [==============================] - 1s 104ms/step - 

In [13]:
test_generator = test_gen.flow_from_dataframe(testset, directory=None, x_col='id', y_col='species', 
                                                target_size=(img_size,img_size), color_mode='grayscale',
                                                class_mode='categorical', batch_size=1)

loss, acc = cnn_model.evaluate_generator(test_generator, steps=len(test_generator))
print('loss:',loss)
print('acc:',acc)

Found 99 images belonging to 99 classes.
loss: 1.271816186981449
acc: 0.5959595959595959
